In [1]:
# %load_ext autoreload
# %autoreload 2

In [2]:
import altair as alt
from coop_finances import *

In [7]:
TAX_RATE = 2.32
ROBBIE_PROPANE = 1135
ROBBIE_ELECTRIC = 1598
ROBBIE_HEATING_OIL = 375




def scenarios(property_cost, interest_rate, insurance, lawyer_fees, clt_value, assessed_value, cords_wood, price_per_cord, maintaince, mobile_home, trailer, propane, electric, oil, loan_duration, bookkeeping, nevins_cabin, storage_shed, per_sq_ft, shipping_container):
    r = interest_rate / 12
    def make_scenario(name, upfront_costs, number_people):
        P = sum(upfront_costs.values())
        return Scenario(
            name,
            monthly_cost={
                "Investors": alt.expr.if_(
                    loan_duration == 0,
                    # Interest only
                    r * P,
                    # Fixed rate repayment
                    # https://en.wikipedia.org/wiki/Mortgage_calculator#Monthly_payment_formula
                    alt.expr.if_(
                        r == 0,
                        P / loan_duration,
                        (r * P) / (1 - (1 + r)**(-loan_duration)),
                    ),
                ),
                "Land Trust": alt.expr.if_(clt_value <= 0, 0, 75),
                "Property Tax": assessed_value * TAX_RATE / 100 / 12,
                "Wood": cords_wood *  price_per_cord / 12,
                "Propane": propane / 12,
                "Electric": electric / 12,
                "Heating Oil": oil / 12,
                "Internet": 99,
                "Maintenance": maintaince / 12,
                "Insurance": insurance / 12,
                "Bookkeeping": bookkeeping
            },
            upfront_cost=upfront_costs,
            number_people=number_people
        )
    return [
        make_scenario("Upfront", {
            "Lawyer Fees": lawyer_fees,
            "Purchase": property_cost - clt_value,
            "Nevin's Cabin - Labor": 500 * 25,
            "Nevin's Cabin - Materials": 5000,
            "Storage Shed + Painting Studio": alt.expr.if_(
                storage_shed == "shipping",
                shipping_container,
                alt.expr.if_(
                    storage_shed == "build",
                    1000 * per_sq_ft,
                    0
                )
            ),
        }, {
            "Mobile Home": mobile_home,
            "Trailer": trailer,
            "Meditation Cabin": alt.expr.if_(nevins_cabin, 1, 0),
            "Storage Shed + Painting Studio": alt.expr.if_(
                storage_shed == "shipping",
                1,
                alt.expr.if_(
                    storage_shed == "build",
                    2,
                    0
                )
            )
        }),
        make_scenario("Renovate Sheds", {
            "Lawyer Fees": lawyer_fees,
            "Purchase": property_cost - clt_value,
            "Nevin's Cabin - Labor": 500 * 25,
            "Nevin's Cabin - Materials": 5000,
            "Storage Shed + Painting Studio": alt.expr.if_(
                storage_shed == "shipping",
                shipping_container,
                alt.expr.if_(
                    storage_shed == "build",
                    1000 * per_sq_ft,
                    0
                )
            ),
            "Renovate Sheds": 100 * 1000
        }, {
            "Mobile Home": mobile_home,
            "Trailer": trailer,
            "Meditation Cabin": alt.expr.if_(nevins_cabin, 1, 0),
            "Storage Shed + Painting Studio": alt.expr.if_(
                storage_shed == "shipping",
                1,
                alt.expr.if_(
                    storage_shed == "build",
                    2,
                    0
                )
            ),
            "Renovated Sheds": 1
        }),
        make_scenario("Build Home", {
            "Lawyer Fees": lawyer_fees,
            "Purchase": property_cost - clt_value,
            "Nevin's Cabin - Labor": 500 * 25,
            "Nevin's Cabin - Materials": 5000,
            "Storage Shed + Painting Studio": alt.expr.if_(
                storage_shed == "shipping",
                shipping_container,
                alt.expr.if_(
                    storage_shed == "build",
                    1000 * per_sq_ft,
                    0
                )
            ),
            "Renovate Sheds": 500 * per_sq_ft,
            "Home": 1500 * per_sq_ft,
        }, {
            "Trailer": trailer,
            "Meditation Cabin": alt.expr.if_(nevins_cabin, 1, 0),
            "Storage Shed + Painting Studio": alt.expr.if_(
                storage_shed == "shipping",
                1,
                alt.expr.if_(
                    storage_shed == "build",
                    2,
                    0
                )
            ),
            "Renovated Sheds": 1,
            "House": 3
        })
    ]

generate_plots(
    scenarios,
    [
        "source: https://github.com/saulshanabrook/coop-finances",
    ],
    mobile_home=Variable("🚌", "# of ppl in mobile home", Range(1, 4, 1), 2, 'O'),
    trailer=Variable("🎬", "# of ppl in studio", Range(0, 4, 1), 2, 'O'),
    nevins_cabin=Variable("🧘", "Meditation Cabin", ([True, False], ("Move Nevin's cabin before sale", "None")), True, 's'),
    storage_shed=Variable("🫙", "Storage Shed + Painting Studio", (["shipping", "build", 0], ("Shipping container before sale", "Build before sale", "None")), 0, 's'),
    shipping_container=Variable("🚢", "Shipping Container Cost", Range(2 * 1000, 10 * 1000, 1000), 7 * 1000, '$'),
    per_sq_ft=Variable("👣", "Construction cost / sq ft", Range(100, 400, 50), 200, '$'),
    property_cost=Variable("🤑", "Purchase price", Range(110 * 1000, 300 * 1000, 10 * 1000), 170 * 1000, '$'),
    interest_rate=Variable("📈", "Investor interest rate", Range(0, 0.10, 0.01), 0.03, '%'),
    loan_duration=Variable("🗓", "Loan Duration", ([5 * 12, 10 * 12, 15 * 12, 30 * 12, 0], ["5 years", "10 years", "15 years", "30 years", "Investment"]), 0, 's'),
    lawyer_fees=Variable("💼", "Lawyer fees and closing costs", Range(2000, 20000, 1000), 10000, '$'),
    clt_value=Variable("🏞", "Land value owned by CLT", Range(0, 150 * 1000, 10000), 70 * 1000, '$'),
    maintaince=Variable("🔧", "Maintenance per year", Range(0.01 * 200 * 1000, 0.05 * 200 * 1000, 0.01 * 200 * 1000), 0.03 * 200 * 1000, '$'),
    assessed_value=Variable("🇺🇸", "Assessed Value (for property taxes)", Range(110 * 1000, 300 * 1000, 5 * 1000), 130 * 1000, '$'),
    bookkeeping=Variable("📔", "Bookkeeping.coop Service Plan", ([230, 460, 1150], ["Extra Small", "Small", "Medium"]), 230, 's'),
    insurance=Variable("🏡", "Home Insurance (annual)", Range(300, 800, 100), 500, '$'),
    cords_wood=Variable("🪵", "Cords of wood per year", Range(0, 5, 1), 3, '#'),
    price_per_cord=Variable("🔥", "Price per cord wood", Range(0, 580, 10), 190, '$'),
    propane=Variable("🦕", "Propane per year", Range(0, ROBBIE_PROPANE * 2, ROBBIE_PROPANE / 10), ROBBIE_PROPANE, '$'),
    electric=Variable("🔌", "Electric per year", Range(0, ROBBIE_ELECTRIC * 2, ROBBIE_ELECTRIC / 10), ROBBIE_ELECTRIC, '$'),
    oil=Variable("🪔", "Heating oil per year", Range(0, ROBBIE_HEATING_OIL * 2, ROBBIE_HEATING_OIL / 10), ROBBIE_HEATING_OIL, '$')
)

alt.HConcatChart(...)